In [1]:
import os
import sys
import torch.utils.data as data
from PIL import Image

class CPN(data.Dataset):
    """
    Args:6
        root (string): Root directory of the VOC Dataset.
        datatype (string): Dataset type 
        image_set (string): Select the image_set to use, ``train``, ``val`` or ``test``
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        dver (str): version of dataset (ex) ``splits/v5/3``
        kfold (int): k-fold cross validation
    """

    def __init__(self, root, datatype='CPN', dver='splits', 
                    image_set='train', transform=None, is_rgb=True):

        self.transform = transform
        self.is_rgb = is_rgb

        image_dir = os.path.join(root, 'CPN_all', 'Images')
        mask_dir = os.path.join(root, 'CPN_all', 'Masks')
        m_image_dir = os.path.join(root, 'Median', 'Images')
        m_mask_dir = os.path.join(root, 'Median', 'Masks')

        if not os.path.exists(image_dir) or not os.path.exists(mask_dir):
            raise Exception('Dataset not found or corrupted.')
        
        split_f = os.path.join(root, 'CPN_all', dver, image_set.rstrip('\n') + '.txt')
        m_split_f = os.path.join(root, 'Median/splits', image_set.rstrip('\n') + '.txt')

        if not os.path.exists(split_f):
            raise Exception('Wrong image_set entered!' 
                            'Please use image_set="train" or image_set="val"', split_f)

        with open(os.path.join(split_f), "r") as f:
            file_names = [x.strip() for x in f.readlines()]
        with open(os.path.join(m_split_f), "r") as f:
            m_file_names = [x.strip() for x in f.readlines()]

        self.image = [os.path.join(image_dir, x + ".bmp") for x in file_names]
        self.mask = [os.path.join(mask_dir, x + "_mask.bmp") for x in file_names]
        self.m_images = [os.path.join(m_image_dir, x + ".jpg") for x in m_file_names]
        self.m_masks = [os.path.join(m_mask_dir, x + ".jpg") for x in m_file_names] 
    
        assert (len(self.image) == len(self.mask))  

        if image_set == 'train' or image_set == 'val':
            self.image.extend(self.m_images)
            self.mask.extend(self.m_masks)

        self.images = []
        self.masks = []

        for index in range(len(self.image)):
            if self.is_rgb:
                img = Image.open(self.image[index]).convert('RGB')
                target = Image.open(self.mask[index]).convert('L')         
            else:
                img = Image.open(self.image[index]).convert('L')
                target = Image.open(self.mask[index]).convert('L')            

            if self.transform is not None:
                img, target = self.transform(img, target)
            
            self.images.append(img)
            self.masks.append(target)
        
        print(f'Images len: {len(self.images)} shape: {self.images[0].shape}')

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is the image segmentation.
        """

        return self.images[index], self.masks[index]

    def __len__(self):
        return len(self.images)    

In [2]:
from utils import ext_transforms as et
from torch.utils.data import DataLoader
from tqdm import tqdm

transform = et.ExtCompose([
        et.ExtRandomCrop(size=(512, 512), pad_if_needed=True),
        et.ExtScale(scale=0.5),
        et.ExtToTensor(),
        et.ExtNormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

dst = CPN(root='/mnt/server5/sdi/datasets', datatype='CPN', image_set='train',
                transform=transform, is_rgb=True, dver='splits/v5/3')
train_loader = DataLoader(dst, batch_size=16,
                            shuffle=True, num_workers=2, drop_last=True)

for epoch in range(100):
    for i, (ims, lbls) in tqdm(enumerate(train_loader)):
        print(f'Epoch {epoch}')
        break
    

Images len: 1366 shape: torch.Size([3, 256, 256])


0it [00:00, ?it/s]

Epoch 0



0it [00:00, ?it/s]

Epoch 1


...

Epoch 97


0it [00:00, ?it/s]


Epoch 98


0it [00:00, ?it/s]


Epoch 99
